<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pip
import sys
#if not 'sklearn' in sys.modules.keys():
#    pip.main(['install', 'sklearn'])
#if not 'kaggle' in sys.modules.keys():
#    pip.main(['install', 'kaggle'])
import random

print("Random number with seed 2020")
# first call
random.seed(2020)

Random number with seed 2020


In [ ]:
import numpy as np
import pandas as pd
import graphviz

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt

# Read the data from GitHub
train = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv')


In [ ]:
#Le classi sono 1=Good 0=Disappointing
train["Quality"] = np.where(train["Quality"].str.contains("Good"), 1, 0)

In [ ]:
# pull data into target (y) and predictors (X)
#La y è la classe, cioè la Quality
train_y = train.Quality
#seleziono colonne di interesse, non metto l'ID perchè non mi interessa
predictor_cols = ['fixed.acidity','volatile.acidity','citric.acid','residual.sugar','chlorides','free.sulfur.dioxide','total.sulfur.dioxide','density','pH','sulphates','alcohol']

# La x sono gli attributi
train_X = train[predictor_cols]

# Sostituisce i missing values con la media e lo applica alle x
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(train_X)


# Impute our data, then train
train_X_imp = imp.transform(train_X)


In [ ]:
#Uso l'80% dei dati per train e il restante 20% per test
xTrain, xTest, yTrain, yTest = train_test_split(train_X_imp, train_y, train_size = 0.8, random_state = 0)

In [ ]:
def generateSubmission(myModel, submissionFile, description):
    # Read the test data
    test = pd.read_csv('https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/test.csv')


    # Treat the test data in the same way as training data. In this case, pull same columns.
    test_X = test[predictor_cols]

    # Impute each test item, then predict
    test_X_imp = imp.transform(test_X)
    
    # Use the model to make predictions
    predicted_q = myModel.predict(test_X_imp)
    # We will look at the predicted Qualities to ensure we have something sensible.
    print(predicted_q)
    
    #submission file
    my_submission = pd.DataFrame({'Id': test.Id, 'Quality': predicted_q})
    # you could use any filename. We choose submission here
    my_submission.to_csv(submissionFile, index=False)
    
    #Submit authomatically; kaggle API authentication needed
    #!kaggle competitions submit -c mldm-classification-competition-2020 -f {submissionFile} -m '{description}'

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
log_reg = LogisticRegression()

#Range di parametri che vengono esplorati per trovare la combinazione migliore
parameter_grid = {'penalty': ['none', 'l2'],
                  'dual':[False],
                  'tol': [1e-4,1e-6,1e-2,1],
                  'C': [0.01,0.1,0.5,2,50],
                  #'class_weight' : [{'alcohol' : [2,3]}], 
                  'solver' : ['saga'],
                  'max_iter': [30,50,100,200]
                  }

cross_validation = StratifiedKFold(n_splits=10)
cross_validation.get_n_splits(train_X_imp, train_y)

#Create the scoring dictionary
SCORING = {'accuracy': 'accuracy',
'balanced_accuracy': 'balanced_accuracy',
'precision': 'precision_macro',
'recall': 'recall_macro',
'f1': 'f1_macro'}

#grid_search = GridSearchCV(dtc, param_grid=parameter_grid, cv=cross_validation)
grid_search = GridSearchCV(log_reg, param_grid=parameter_grid, cv=cross_validation, scoring=SCORING,return_train_score=True, refit='f1')

grid_search.fit(train_X_imp, train_y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio para

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprec...
             param_grid={'C': [0.01, 0.1, 0.5, 2, 50], 'dual': [False],
                         'max_iter': [30, 50, 100, 200],
                         'penalty': ['none', 'l2'], 'solver': ['saga'],
                         'tol': [0.0001, 1e-06, 0.01, 1]},
             pre_dispatch='2*n_job

In [ ]:
grid_search.cv_results_

{'mean_fit_time': array([0.03371334, 0.03336709, 0.03374279, 0.00373018, 0.03263218,
        0.03382306, 0.03499923, 0.00355995, 0.05300667, 0.05303652,
        0.05363491, 0.00406272, 0.0524971 , 0.05292106, 0.05229468,
        0.00351701, 0.10289798, 0.10326591, 0.0617779 , 0.00379496,
        0.10207679, 0.1035094 , 0.06012487, 0.00356233, 0.16397307,
        0.16380112, 0.06094382, 0.00388141, 0.16384208, 0.16320212,
        0.06061761, 0.00347598, 0.03372447, 0.03301525, 0.03233104,
        0.0038852 , 0.03299274, 0.03319688, 0.03270473, 0.00336428,
        0.0523756 , 0.0539166 , 0.05351813, 0.00389726, 0.05392451,
        0.0527267 , 0.05236979, 0.00345597, 0.101425  , 0.10291495,
        0.06118402, 0.00384386, 0.10192311, 0.10222893, 0.06151118,
        0.00341492, 0.1652411 , 0.16950111, 0.06079588, 0.0042486 ,
        0.16427131, 0.16364727, 0.05964935, 0.00358136, 0.03283069,
        0.03355167, 0.03337252, 0.00385849, 0.03364131, 0.03246799,
        0.03390985, 0.00400422,

In [ ]:
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_dtc = grid_search.best_estimator_
best_dtc

Best score: 0.5362064362643515
Best parameters: {'C': 0.1, 'dual': False, 'max_iter': 200, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.0001}


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
my_model=best_dtc
my_model.fit(train_X_imp, train_y)
my_model.score(train_X_imp, train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6638005159071367

In [ ]:
y_pred = my_model.predict(train_X_imp)
print("Confusion Matrix:")
print(confusion_matrix(train_y, y_pred))

print("Classification Report:")
print(classification_report(train_y, y_pred))

Confusion Matrix:
[[ 247  958]
 [ 215 2069]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.20      0.30      1205
           1       0.68      0.91      0.78      2284

    accuracy                           0.66      3489
   macro avg       0.61      0.56      0.54      3489
weighted avg       0.63      0.66      0.61      3489



In [ ]:
# The snippet below will retrieve the feature importances from the model and make them into a DataFrame.
feature_importances = pd.DataFrame(my_model.feature_importances_,
                                   index = train_X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

AttributeError: ignored

In [ ]:
# generate a submission file
generateSubmission(my_model,'LogisticRegression2.csv', "User defined logistic regression evaluated with Grid Search and SCORE")

[0 1 1 ... 1 1 1]
